<div style='float: right'><img src='pic/sudoku.png'/></div>

## <div id='sudoku' />数独

In [1]:
#!pip3 install pulp ortoolpy
import numpy as np, pandas as pd
from pulp import LpProblem, lpDot, lpSum, value
from itertools import product
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
..6.....1
.7..6..5.
8..1.32..
..5.4.8..
.4.7.2.9.
..8.1.7..
..12.5..3
.6..7..8.
2.....4..""".replace('\n', '')

### 問題
* マスに 1～9 の数字を入れます
* 縦、横または3×3に同じ数字は入りません

### 定式化
\begin{array}{cl}
            変数 & v_{ijk} \in \{0, 1\} ~ \forall i, j, k ~ ~ ~ マスi,jが数字k+1か (1) \\
\mbox{subject to} & \sum_k{v_{ijk}} = 1 ~ \forall i, j ~数字は1つ (2) \\
                 & \sum_k{v_{ikj}} = 1 ~ \forall i, j ~ 縦に同じ数字はない (3) \\
                 & \sum_k{v_{kij}} = 1 ~ \forall i, j ~ 横に同じ数字はない (4) \\
                 & 3\times3のマスについても同様 (5) \\
                 & 数字指定 (6) \\
\end{array}

In [2]:
r = range(9)
a = pd.DataFrame([(i,j,(i//3)*3+j//3,k+1,c==str(k+1))
    for (i,j),c in zip(product(r,r),data) for k in r],
    columns=['行','列','_3x3','数','固'])
a['Var'] = addbinvars(len(a))
m = LpProblem()
for cl in [['行','列'],['行','数'],['列','数'],['_3x3','数']]:
    for _,v in a.groupby(cl):
        m += lpSum(v.Var) == 1
for _,r in a[a.固==True].iterrows():
    m += r.Var == 1
m.solve() # ソルバーで求解
a['Val'] = a.Var.apply(value)
a[a.Val>0.5].数.values.reshape(9,9)

array([[5, 3, 6, 8, 2, 7, 9, 4, 1],
       [1, 7, 2, 9, 6, 4, 3, 5, 8],
       [8, 9, 4, 1, 5, 3, 2, 6, 7],
       [7, 1, 5, 3, 4, 9, 8, 2, 6],
       [6, 4, 3, 7, 8, 2, 1, 9, 5],
       [9, 2, 8, 5, 1, 6, 7, 3, 4],
       [4, 8, 1, 2, 9, 5, 6, 7, 3],
       [3, 6, 9, 4, 7, 1, 5, 8, 2],
       [2, 5, 7, 6, 3, 8, 4, 1, 9]])